# TME 2 : Prédiction de gènes - Détection de gènes candidats

Le but de ce TME est d’annoter les régions dans le génome qui correspondent à des gènes. Nous allons prendre en compte les différents éléments qui définissent un gène pour pouvoir déterminer les candidats à des régions codantes : phases ouvertes de lecture, propriétés statistiques du code génétique, et comparer les résultats avec l’annotation qui est disponible. 
Nous nous baserons sur le génome de _Bifidobacterium actinocoloniiforme_ que nous avons commencé à analyser au TME précédent, avec ses annotations et au génome de _Escherichia coli_


### A) Mise en route,  télécharger et preparer les données

<b>Question 1)</b> Adapter les commandes du TME précédent pour télécharger le génome de _E. coli_ ainsi que ses annotations


ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/026/345/GCF_000026345.1_ASM2634v1/GCF_000026345.1_ASM2634v1_genomic.fna.gz

ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/026/345/GCF_000026345.1_ASM2634v1/GCF_000026345.1_ASM2634v1_feature_table.txt.gz


In [3]:
import urllib.request
#Utliser la fonction urllib.request.urlretrieve(path)

In [4]:
#download _genomic.fna.gz
urllib.request.urlretrieve("ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/026/345/GCF_000026345.1_ASM2634v1/GCF_000026345.1_ASM2634v1_genomic.fna.gz","_genomic.fna.gz")

('_genomic.fna.gz', <http.client.HTTPMessage at 0x7f9884236da0>)

In [5]:
#download _feature_table.txt.gz
urllib.request.urlretrieve("ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/026/345/GCF_000026345.1_ASM2634v1/GCF_000026345.1_ASM2634v1_feature_table.txt.gz","_feature_table.txt.gz")

('_feature_table.txt.gz', <http.client.HTTPMessage at 0x7f98841e4080>)

<b>Question 2)</b> Decompresser les fichiers

In [6]:
import gzip
import shutil

In [7]:
#Decompresser _genomic.fna.gz
%%bash
gunzip _genomic.fna.gz

SyntaxError: invalid syntax (<ipython-input-7-ed2cabc5c73a>, line 3)

In [8]:
#Decompresser _feature_table.txt.gz
%%bash
gunzip _feature_table.txt.gz

SyntaxError: invalid syntax (<ipython-input-8-eff64ae05575>, line 3)

<b>Question 3)</b> Exécuter le code ci-dessous qui va créer un dictionnaire pour représenter le code genetique. Vous avez également deux jeu de séquences pour tester les prochaines fonctions.

In [9]:
# Dictionnaire codons -> acides aminés. Les codons stops sont représentés
# avec le caractère "*"
CODEGENETIQUE = {
    "TTT": "F",
    "TTC": "F",
    "TTA": "L",
    "TTG": "L",
    "TCT": "S",
    "TCC": "S",
    "TCA": "S",
    "TCG": "S",
    "TAT": "Y",
    "TAC": "Y",
    "TAA": "*",
    "TAG": "*",
    "TGT": "C",
    "TGC": "C",
    "TGA": "*",
    "TGG": "W",
    "CTT": "L",
    "CTC": "L",
    "CTA": "L",
    "CTG": "L",
    "CCT": "P",
    "CCC": "P",
    "CCA": "P",
    "CCG": "P",
    "CAT": "H",
    "CAC": "H",
    "CAA": "Q",
    "CAG": "Q",
    "CGT": "R",
    "CGC": "R",
    "CGA": "R",
    "CGG": "R",
    "ATT": "I",
    "ATC": "I",
    "ATA": "I",
    "ATG": "M",
    "ACT": "T",
    "ACC": "T",
    "ACA": "T",
    "ACG": "T",
    "AAT": "N",
    "AAC": "N",
    "AAA": "K",
    "AAG": "K",
    "AGT": "S",
    "AGC": "S",
    "AGA": "R",
    "AGG": "R",
    "GTT": "V",
    "GTC": "V",
    "GTA": "V",
    "GTG": "V",
    "GCT": "A",
    "GCC": "A",
    "GCA": "A",
    "GCG": "A",
    "GAT": "D",
    "GAC": "D",
    "GAA": "E",
    "GAG": "E",
    "GGT": "G",
    "GGC": "G",
    "GGA": "G",
    "GGG": "G"
}

# Utiliser les sequences TESTSEQ et TESTSEQCLEAN pour tester vos fonctions
TESTSEQ = "ATGAAACGCATTAGCMMCACCATTACCACCACCATCACCATTACCACAGKTAACGGTGCGGGCTGA"
TESTSEQCLEAN = "ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA"

### B) Annotation des phases ouvertes de lecture et traduction en séquence protéique

<b>Question 1)</b> Parfois dans les génomes (représentés par des fichiers numériques), nous trouvons des nucléotides qui n'ont pas été correctement identifiés par la machine de séquençage. Certains instruments nous restreignent les possibilités à un sous-ensemble de nucléotides. Ecrire une fonction `remplace_non_identifies` qui remplace les nucléotides non identifiés par une des possibilités listées ci-dessous de façon aléatoire.<br>

R = G,A (purine)<br>
Y = T,C (pyrimidine)<br>
K = G,T (céto)<br>
M = A,C (amino)<br>
S = G,C (obligations solides)<br>
W = A,T (Les liaisons faibles)<br>
B = G,T,C (tous sauf A)<br>
D = G,A,T (tous sauf C)<br>
H = A,C,T (tous sauf G)<br>
V = G,C,A (tous sauf T)<br>
N = A,G,C,T (any)<br>
X = A,G,C,T (any)<br>



In [10]:
import doctest # C’est pour pouvoir utiliser doctest.testmod() et tester les fonctions
import random

In [11]:
random.choice(['A', 'C', 'T', 'G'])

'T'

In [12]:
##### Question 1
def remplace_non_identifies(seq):
    """
    Remplace les nucléotides non identifiés par une des possibilités de façon aléatoire.
    
    """
    options = {
        "R": ["G", "A"],
        "Y": ["T", "C"],
        "K": ["G", "T"],
        "M": ["A", "C"],
        "S": ["G", "C"],
        "W": ["A", "T"],
        "B": ["G", "T", "C"],
        "D": ["G", "A", "T"],
        "H": ["A", "C", "T"],
        "V": ["G", "C", "A"],
        "N": ["A", "G", "C", "T"],
        "X": ["A", "G", "C", "T"]
    }
    
    seq2 = ""
    
    for i in range(len(seq)):
        if(seq[i] in options.keys()):
            seq2 += random.choice(options[seq[i]])
        else:
            seq2 += seq[i]
    
    return seq2


doctest.testmod()

remplace_non_identifies(TESTSEQ)

**********************************************************************
File "__main__", line 5, in __main__.liste_orfs2
Failed example:
    sorted(liste_orfs('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))
Exception raised:
    Traceback (most recent call last):
      File "/usr/local/anaconda3/lib/python3.7/doctest.py", line 1329, in __run
        compileflags, 1), test.globs)
      File "<doctest __main__.liste_orfs2[0]>", line 1, in <module>
        sorted(liste_orfs('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))
    NameError: name 'liste_orfs' is not defined
**********************************************************************
1 items had failures:
   1 of   1 in __main__.liste_orfs2
***Test Failed*** 1 failures.


'ATGAAACGCATTAGCCACACCATTACCACCACCATCACCATTACCACAGTTAACGGTGCGGGCTGA'

<b>Question 2)</b> Ecrire une fonction `listecodon` qui renvoie une liste de codons pour une séquence passée en paramètre. Par exemple, si on passe la séquence ``AACGTGGCA`` comme paramètre votre fonction doit renvoyer ``[‘AAC’, ‘GTG’, ‘GCA’]``. Si la longueur de la séquence n'est pas un multiple de 3 on ne tiendra pas compte des 1 ou 2 nucléotides restant à la fin.


In [13]:
?range

In [14]:
# Question 2
def listecodon(seq):
    """
    Renvoie une liste de codons pour une séquence passée en paramètre.
    
    Si la longueur de la séquence n'est pas un multiple de 3 elle ne tiendra pas 
    compte des 1 ou 2 nucléotides restant à la fin.
    
    >>> listecodon('AAACCC')
    ['AAA', 'CCC']
    >>> listecodon('AAACC')
    ['AAA']
    >>> listecodon('AAAC')
    ['AAA']
    """
    res = []
    
    for i in range(0, len(seq), 3) :
        if i+3 <= len(seq) :
            res.append(seq[i:i+3])
    return res

doctest.testmod()

assert listecodon(TESTSEQCLEAN) == [
    'ATG', 'AAA', 'CGC', 'ATT', 'AGC', 'ACC', 'ACC', 'ATT', 'ACC', 'ACC',
    'ACC', 'ATC', 'ACC', 'ATT', 'ACC', 'ACA', 'GGT', 'AAC', 'GGT', 'GCG',
    'GGC', 'TGA'
]

**********************************************************************
File "__main__", line 5, in __main__.liste_orfs2
Failed example:
    sorted(liste_orfs('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))
Exception raised:
    Traceback (most recent call last):
      File "/usr/local/anaconda3/lib/python3.7/doctest.py", line 1329, in __run
        compileflags, 1), test.globs)
      File "<doctest __main__.liste_orfs2[0]>", line 1, in <module>
        sorted(liste_orfs('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))
    NameError: name 'liste_orfs' is not defined
**********************************************************************
1 items had failures:
   1 of   1 in __main__.liste_orfs2
***Test Failed*** 1 failures.


<b>Question 3)</b> Ecrire une fonction `reversecompl` qui renvoie le brin complémentaire d’une séquence passée en paramètre. Par exemple, si on passe la séquence ``AACGTGGCA`` comme paramètre votre fonction doit renvoyer ``TGCCACGTT``.


In [15]:
# Question 3
def reversecompl(seq):
    """Renvoie le brin complémentaire d’une séquence."""
    compl = {'A': 'T', 'C': 'G', 'G': 'C', 'T':'A'}
    letters = [compl[seq[i]] for i in range(len(seq))]
    return (''.join(letters))[::-1]
    


#print (reversecompl('AACGTGGCA'))
assert reversecompl(
    TESTSEQCLEAN
) == "TCAGCCCGCACCGTTACCTGTGGTAATGGTGATGGTGGTGGTAATGGTGGTGCTAATGCGTTTCAT"

<b>Question 4)</b> Ecrire une fonction `liste_orfs` qui retourne la liste de tous les cadres ouverts de lectures, c'est à dire les séquence commençant par un des codons start: ``ATG``, ``GTG``, ``TTG`` (``ATG`` est le plus fréquent) et finissant par un des codons stop : ``TAA``, ``TAG``, ``TGA``. Vous renverrez les séquences pour les 6 phases de lecture (3 pour le brin sens et 3 pour le brin complémentaire). Ces séquences sont nommées séquences CDS (pour _CoDing Sequences_ en anglais) et les phases ouvertes de lecture ORFS (pour _Open Reading Frame_). <br>
Note 1 : si plusieurs start sont trouvés dans la même phase de lecture qu'un stop et en amont de celui ci, le CDS débute par conventions au start le plus en amont. <br>
Note 2 : les CDS peuvent se chevaucher (sur différentes phases et éventuellement sur le brin d'ADN complémentaire).


In [16]:
# Question 4
def __trouver_orfs(codons):
    """Retourne cadres ouverts dans la liste des codons."""
    starts = [
        i for i, codon in enumerate(codons) if codon in {'ATG', 'GTG', 'TTG'}
    ]
    stops = [
        i for i, codon in enumerate(codons) if codon in {'TAA', 'TAG', 'TGA'}
    ]
    
    if starts and stops:
        debut = starts[0]
        fin = stops[0]
    else:
        debut = len(starts) - 1
        fin = len(starts) - 2
    
    return (debut, fin)

def liste_orfs(seq):
    """
    Retourne la liste de tous les cadres ouverts de lectures.
        
    >>> sorted(liste_orfs('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))
    ['ATGATGTAA', 'ATGCCCTAA', 'GTGTTTTGA']
    """
    codon_seq1 = listecodon(seq)
    codon_seq2 = listecodon(seq[1:])
    codon_seq3 = listecodon(seq[2:])
    
    
    orf1_debut, orf1_fin = __trouver_orfs(codon_seq1)
    orf2_debut, orf2_fin = __trouver_orfs(codon_seq2)
    orf3_debut, orf3_fin = __trouver_orfs(codon_seq3)
    
    
    
    seq_tmp = ""
    orfs = []
    
    for index in range(orf1_debut, orf1_fin + 1):
        seq_tmp += codon_seq1[index]
        
    if len(seq_tmp) != 0:
        orfs.append(seq_tmp)
    seq_tmp = ""
    
    
    
    
    for index in range(orf2_debut, orf2_fin + 1):
        seq_tmp += codon_seq2[index]
    
    if len(seq_tmp) != 0:
        orfs.append(seq_tmp)
    seq_tmp = ""
    
    
    
    
    for index in range(orf3_debut, orf3_fin + 1):
        seq_tmp += codon_seq3[index]
    
    if len(seq_tmp) != 0:
        orfs.append(seq_tmp)
    seq_tmp = ""
    
    
    
    seq_reverse = reversecompl(seq)
    
    codon_seq1_reverse = listecodon(seq_reverse)
    codon_seq2_reverse = listecodon(seq_reverse[1:])
    codon_seq3_reverse = listecodon(seq_reverse[2:])
    
    
    orf1_debut_reverse, orf1_fin_reverse = __trouver_orfs(codon_seq1_reverse)
    orf2_debut_reverse, orf2_fin_reverse = __trouver_orfs(codon_seq2_reverse)
    orf3_debut_reverse, orf3_fin_reverse = __trouver_orfs(codon_seq3_reverse)
    
    
    seq_reverse_tmp = ""
    
    for index in range(orf1_debut_reverse, orf1_fin_reverse + 1):
        seq_reverse_tmp += codon_seq1_reverse[index]
        
    if len(seq_reverse_tmp) != 0:
        orfs.append(seq_reverse_tmp)
    seq_reverse_tmp = ""
    
    
    
    
    for index in range(orf2_debut_reverse, orf2_fin_reverse + 1):
        seq_reverse_tmp += codon_seq2_reverse[index]
        
    if len(seq_reverse_tmp) != 0:
        orfs.append(seq_reverse_tmp)
    seq_reverse_tmp = ""
    
    
    
    
    for index in range(orf3_debut_reverse, orf3_fin_reverse + 1):
        seq_reverse_tmp += codon_seq3_reverse[index]
        
    if len(seq_reverse_tmp) != 0:
        orfs.append(seq_reverse_tmp)
    seq_reverse_tmp = ""
    

    

    return orfs


doctest.testmod()

TestResults(failed=0, attempted=5)

In [17]:
 liste_orfs(TESTSEQCLEAN)

['ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA']

Voici un code efficaz qui utilise Regular Expression

In [18]:
import re

In [19]:
[
    match.group()
    for match in re.finditer('((ATG|GTG|TTG)((?!TAA|TAG|TGA)([ACTG][ACTG][ACTG]))*(TAA|TAG|TGA))',
                             'AAATGATGTAATAGTGTTTTGATTAGGGCAT')
]

['ATGATGTAA', 'GTGTTTTGA']

In [20]:
def __liste_orfs_sens(seq):
    """Retourne la liste des cadres ouverts de lecture, sens 5' vers 3'."""
    return [
        match.group() for match in re.finditer(
            '((ATG|GTG|TTG)((?!TAA|TAG|TGA)([ACTG][ACTG][ACTG]))*(TAA|TAG|TGA))',
            seq)
    ]


def liste_orfs(seq):
    """
    Retourne la liste de tous les cadres ouverts de lectures.
        
    >>> sorted(liste_orfs('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))
    ['ATGATGTAA', 'ATGCCCTAA', 'GTGTTTTGA']
    """
    liste = __liste_orfs_sens(seq)
    liste.extend(__liste_orfs_sens(reversecompl(seq)))
    return liste


doctest.testmod()

TestResults(failed=0, attempted=5)

In [21]:
liste_orfs(TESTSEQCLEAN)

['ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA']

<b>Question 5)</b> Développez une fonction `traduit_prot` qui étant donné un gène renvoie la protéine traduite. Utilisez le dictionnaire définit ci dessous.

In [22]:
# Question 5
def traduit_prot(seq, codegenetique=CODEGENETIQUE):
    """Renvoie la protéine traduite, par défaut il utilise CODEGENETIQUE."""
    
    proteine = ""
    
    codon_du_seq = listecodon(seq)
    for codon in codon_du_seq:
        proteine += codegenetique[codon]
    
    return proteine


assert traduit_prot(TESTSEQCLEAN) == "MKRISTTITTTITITTGNGAG*"

### C) Analyse des phases ouvertes de lecture

<b>Question 1)</b> Utilisez votre fonction `liste_orfs` qui trouve toutes les ORFs pour ne trouver que les ORFs d’un génome sur le brin sens, c'est à dire pour trois premières phases. Sauvegarder l'ensemble des ORFs trouvé dans un fichier de type fasta.


In [23]:
def read_fasta(filename):
    """Lire un fichier fasta dans un dict à partir des identifiants des séquences."""
    seqs = {}
    with open(filename, 'r') as file:
        seqid = ''
        seq = ''
        for line in file:
            if line.startswith('>'):
                if seqid != '':
                    seqs[seqid] = seq
                else:
                    seq = ''
                seqid = line.replace('>', '').strip()
            else:
                seq = seq + line.strip()
        seqs[seqid] = seq

    return seqs

In [24]:
def write_fasta(filename, sequences):
    """Ecrivez un fichier fasta à partir d'une liste de séquences."""
    with open(filename, 'w') as file:
        counter = 0
        for seq in sequences:
            file.write('>seq_{}\n{}\n'.format(counter, seq))
            counter += 1

In [25]:
def head(filename):
    """Montrer les 10 premières lignes."""
    with open(filename) as file:
        counter = 0
        for line in file:
            print(line.strip())
            counter += 1
            if counter == 10:
                break

In [26]:
def liste_orfs2(seq):
    """
    Retourne la liste de tous les cadres ouverts de lectures.
        
    >>> sorted(liste_orfs('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))
    ['ATGATGTAA', 'ATGCCCTAA', 'GTGTTTTGA']
    """
    codon_seq1 = listecodon(seq)
    codon_seq2 = listecodon(seq[1:])
    codon_seq3 = listecodon(seq[2:])
    
    
    orf1_debut, orf1_fin = __trouver_orfs(codon_seq1)
    orf2_debut, orf2_fin = __trouver_orfs(codon_seq2)
    orf3_debut, orf3_fin = __trouver_orfs(codon_seq3)
    
    
    
    seq_tmp = ""
    orfs = []
    
    for index in range(orf1_debut, orf1_fin + 1):
        seq_tmp += codon_seq1[index]
        
    if len(seq_tmp) != 0:
        orfs.append(seq_tmp)
    seq_tmp = ""
    
    
    
    
    for index in range(orf2_debut, orf2_fin + 1):
        seq_tmp += codon_seq2[index]
    
    if len(seq_tmp) != 0:
        orfs.append(seq_tmp)
    seq_tmp = ""
    
    
    
    
    for index in range(orf3_debut, orf3_fin + 1):
        seq_tmp += codon_seq3[index]
    
    if len(seq_tmp) != 0:
        orfs.append(seq_tmp)
    seq_tmp = ""
    
    
    return orfs
print(liste_orfs2('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))

['GTGTTTTGA', 'ATGATGTAA']


In [27]:
sequence = 'AAATGATGTAATAGTGTTTTGATTAGGGCAT'

for s in liste_orfs2(sequence):
    write_fasta("exoC.fasta", s)


<b>Question 2)</b> Dessinez un histogramme (avec matplotlib) de la distribution de longueur des ORFs détectés précédemment, que remarquez vous ? 


In [31]:
# Question 2

# Plot avec matplotlib
import matplotlib.pyplot as plt





<b>Question 3)</b> Utilisez la question précédente pour fixer un seuil pour dire qu’un ORF est un gène. (par exemple plus que 99 nucléotides). En déduire une première annotation des gènes dans le génome de _Escherichia coli_. Vous produirez un fichier tab avec les positions de tous les ORFs annotés. Quels gènes du brin positif vont être ratés à votre avis ? 

In [29]:
# Question 3
# Fichier d'annotation des ORFs
